# Objetivo del Taller 01
Que los estudiantes pongan en práctica y refuercen lo aprendido en las sesiones 2 y 3, aplicando técnicas de ingesta de datos, validación de esquemas, manejo de errores y registro de auditoría, utilizando un dataset de origen real o simulado.

### 1. Escoger un dataset de origen (puede ser un archivo CSV, JSON o Parquet disponible en un volumen de Unity Catalog o cargado manualmente).

Se suben los archivos a la ruta /Volumes/taller_evualado_01/default/taller01/input/

### 2. Diseñar y ejecutar un flujo de ingesta hacia Databricks utilizando Auto Loader según el formato y el volumen del archivo. (primero va el paso 3, luego el paso 2)

In [0]:
from pyspark.sql.functions import col

df_compras = (
    spark.read
    .format("json")
    .schema(compras_schema)
    .load("/Volumes/taller_evualado_01/default/taller01/input/")
    .withColumn("archivo_origen", col("_metadata.file_path"))
)

In [0]:
display(df_compras)

### 3. Definir un esquema explícito con StructType y aplicarlo durante la carga.

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, TimestampType

compras_schema = (
    StructType()
    .add("compra_id", StringType())
    .add("cliente_id", StringType())
    .add("producto", StringType())
    .add("cantidad", IntegerType())
    .add("precio_unitario", DoubleType())
    .add("fecha_compra", TimestampType())
)

### 4. Configurar un badRecordsPath y capturar información de registros inválidos.

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, TimestampType

compras_schema = (
    StructType()
    .add("compra_id", StringType())
    .add("cliente_id", StringType())
    .add("producto", StringType())
    .add("cantidad", IntegerType())
    .add("precio_unitario", DoubleType())
    .add("fecha_compra", TimestampType())
    .add("_rescued_data", StringType())
)

In [0]:
from pyspark.sql.functions import col

df_compras = (
    spark.read
    .format("json")
    .schema(compras_schema)
    .option("columnNameOfCorruptRecord", "_rescued_data")
    #.option("badRecordsPath", "/Volumes/taller_evualado_01/default/taller01/bad_records/") # _rescued_data se almacena en un directorio
    .load("/Volumes/taller_evualado_01/default/taller01/input/")
    .withColumn("archivo_origen", col("_metadata.file_path"))
)

In [0]:
display(df_compras)

### 5. Separar datos válidos e inválidos en tablas Delta diferentes.

In [0]:
from pyspark.sql.functions import col

try:
    df_validos = df_compras.filter(col("_rescued_data").isNull())
    df_invalidos = df_compras.filter(col("_rescued_data").isNotNull())

    df_validos.write.format("delta").mode("overwrite").saveAsTable("taller_evualado_01.default.compras_bien")
    df_invalidos.write.format("delta").mode("overwrite").saveAsTable("taller_evualado_01.default.compras_mal")

    print("Escritura realizada con éxito.")

except Exception as e:
    print(f"Error durante la escritura: {str(e)}")

In [0]:
%sql  select * from taller_evualado_01.default.compras_bien

In [0]:
%sql  select * from taller_evualado_01.default.compras_mal

### 6. Agregar una columna de auditoría con la ruta de archivo de origen (_metadata.file_path).

In [0]:
from pyspark.sql.functions import col

df_compras = (
    spark.read
    .format("json")
    .schema(compras_schema)
    .option("columnNameOfCorruptRecord", "_rescued_data")
    #.option("badRecordsPath", "/Volumes/taller_evualado_01/default/taller01/bad_records/") # _rescued_data se almacena en un directorio
    .load("/Volumes/taller_evualado_01/default/taller01/input/")
    .withColumn("auditoria", col("_metadata.file_path"))
)

In [0]:
display(df_compras)

### 7. Guardar los registros inválidos en una tabla Delta de auditoría usando saveAsTable().

In [0]:
from pyspark.sql.functions import col

df_invalidos = df_compras.filter(col("_rescued_data").isNotNull())
df_invalidos.write.format("delta").mode("overwrite").saveAsTable("taller_evualado_01.default.auditoria_compras")

In [0]:
%sql  select * from taller_evualado_01.default.auditoria_compras

### 8. Documentar las decisiones tomadas, las dificultades encontradas y los resultados obtenidos.

1. Desiciones Tomadas
   
    a. Se definio la data a ingestar y luego usar la opcion de read y no readstream
    
    b. usar las clases para dar forma al notebook

2. Dificultades encontradas
    
    a. una dificultad mia fue el conocer poco el tema de databricks

3. Resultados Obtenidos
    
    a. aprender mas sobre el uso de databricks
   
    b. aprender sobre la ingesta de datos
    
    c. aprender a separar los registros malos de los buenos
   
    d. guardar en tablas distinas los registros buenos de los malos

Imagenes de las tablas Delta

![](/Volumes/taller_evualado_01/default/taller01/images/tablas delta.jpg)
![](/Volumes/taller_evualado_01/default/taller01/images/tabla_compras_bien.jpg)
![](/Volumes/taller_evualado_01/default/taller01/images/tabla_compras_mal.jpg)
![](/Volumes/taller_evualado_01/default/taller01/images/tabla_auditoria_compras.jpg)